---
# 1. Collecting data using YouTube api.
---

### 1. a) Importing required libraries:-

In [83]:
import os
import random
import requests
import numpy as np
import pandas as pd
import isodate as iso
import psycopg2 as spg
from io import StringIO
from pprint import pprint
from dotenv import load_dotenv

### 1. b) Requirements for API call:-
1. Loading api key (credential) stored as envirnment variable.
1. Creating efficient HTTP request following the official YouTube API V3 Documentation.

- Loading api key

In [84]:
# If there is/will be no privacy issue, you may assign the API key normally in this file like: api_key = "[YOUR API KEY]".

load_dotenv(dotenv_path=".env") # in case of multiple environments, it's better to mention the path
api_key = os.getenv("API_KEY")

- Building the URL for the api request. (PS: <u>[Official Documentation](developers.google.com/youtube/v3/docs "YouTube Data API Reference")</u>)

In [85]:
# Using multiline representation of very long single-line URL (without page token) for better readability of parameters.
region_code = "IN"   # As per documentation, follow ISO 3166-1 alpha-2 country code. For India, it is "IN".
url1 = ("https://youtube.googleapis.com/youtube/v3/videos?"
       "key="+api_key+
       "&part=snippet%2CcontentDetails%2Cstatistics%2Cstatus%2CliveStreamingDetails%2CtopicDetails"
       "&chart=mostPopular"
       "&maxResults=50"
#      "&pageToken="  This token value is not required for the 1st call, we will update it later for each subsequent api calls.
       "&regionCode="+region_code+
       "&fields="
                "items("
                       "id%2C%20"
                       "snippet(publishedAt%2C%20channelId%2C%20title%2C%20channelTitle%2C%20tags%2C%20categoryId%2C%20defaultLanguage%2C%20defaultAudioLanguage)%2C%20"
                       "contentDetails(duration%2C%20definition%2C%20caption)%2C%20"
                       "status(embeddable%2C%20madeForKids)%2C%20"
                       "statistics(viewCount%2C%20likeCount%2C%20dislikeCount%2C%20commentCount)%2C%20"
                       "topicDetails%2FtopicCategories%2C%20"
                       "liveStreamingDetails(actualStartTime%2CactualEndTime%2CscheduledStartTime%2CscheduledEndTime%2CconcurrentViewers))%2C%20"
                "nextPageToken%2C%20"
                "pageInfo"
)
# Print out "url" to get the single-line URL for the api call (usable in any web browsers like chrome, etc.)

### 1. c) Calling the api, storing the response and updating the url to call more data subsequently & storing the timestamp of data collection:-
<u>**Note:-**</u> There is a **limitation** of max. no. of videos per page, fetched by the HTTP request (currently **50** out of total results of around 200 *[this can be obtained from the **"pageInfo"** property of the result]*). So to get more videos, we need to update the ***"pageToken"*** parameter of the HTTP request URL and set its value same as the value of ***"nextPageToken"*** property of the previous result data as mentioned in the <u>[Official Pagination Guide](https://developers.google.com/youtube/v3/guides/implementation/pagination "Pagination Implementation")</u>. Then we need to call the api again using updated URL. Please refer to the <u>[Official Documentation](https://developers.google.com/youtube/v3/docs "YouTube Data API Reference")</u> for more information.

In [86]:
data1 = requests.get(url1).json()
page2_token = data1["nextPageToken"]
url2 = url1+"&pageToken="+page2_token
data2 = requests.get(url2).json()
# We are stopping here as we already got 100 results.
# But if anyone wants more data, they can continue updating "ulr1" to "url[no. of page]" with the "nextPageToken" property of the last result.
# one can ideally create a loop where results can be fetched in chunks of 50 until it reaches the value of "totalResults" of "pageInfo" property.

# Creating a variable that stores the time zone aware timestamp of the data collection:-
current_timestamp = pd.Timestamp.utcnow()
current_timestamp

Timestamp('2021-09-20 17:57:33.083999+0000', tz='UTC')

### 1. d) Merging the data obtained from each page:-

In [87]:
data = data1["items"]+data2["items"]
print(type(data))
print(len(data))

<class 'list'>
100


### 1. e) Inspecting a sample data to better understand the schema of entire result:-

In [88]:
# Using "Pretty Print" for better readability.
pprint(data[random.randint(0,99)])

{'contentDetails': {'caption': 'true',
                    'definition': 'hd',
                    'duration': 'PT2M43S'},
 'id': 'zu-SP-q9pYU',
 'snippet': {'categoryId': '20',
             'channelId': 'UC4AB0_ectRryjCF_ugD0U8w',
             'channelTitle': 'Free Fire India Official',
             'defaultAudioLanguage': 'en',
             'publishedAt': '2021-09-16T06:30:07Z',
             'tags': ['Free Fire',
                      'Free Fire India',
                      'Free Fire Anniversary',
                      'Free Fire Esports',
                      'Free Fire Moco Awakening',
                      'Free Fire Moco Rebirth',
                      'Moco Awakening',
                      'Moco Rebirth',
                      'Moco',
                      'Free Fire Moco',
                      'Free Fire Characters',
                      'Free Fire Best',
                      'garena free fire'],
             'title': 'Moco Awakens - The Ultimate Battle | Garena Free Fir

---
# 2. Storing Data into Pandas DataFrame object for easier processing.
---

### 2. a) Using ***`pd.json_normalize()`*** [ <u>[Documentation-1](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.io.json.json_normalize.html "pandas 0.22.0")</u> , <u>[Documentation-2](https://pandas.pydata.org/docs/user_guide/io.html?#normalization "pandas 1.3.1")</u> ] to create a Pandas DataFrame from the nested data of api response:-

In [89]:
temp_df = pd.json_normalize(data)
temp_df.head(2)

,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.channelTitle,snippet.tags,snippet.categoryId,contentDetails.duration,contentDetails.definition,contentDetails.caption,...,statistics.viewCount,statistics.likeCount,statistics.dislikeCount,statistics.commentCount,topicDetails.topicCategories,snippet.defaultAudioLanguage,snippet.defaultLanguage,liveStreamingDetails.actualStartTime,liveStreamingDetails.actualEndTime,liveStreamingDetails.scheduledStartTime
0,h0vmsIhQud0,2021-09-20T06:30:04Z,UCx6F-rETGiz7xf_vkMmX2yQ,If Baburao Plays GTA 5!,Mythpat,"[gta 5 baburao, gta 5 funny video, baburao myt...",20,PT13M2S,hd,false,...,2502218,392093,4483,19455,[https://en.wikipedia.org/wiki/Action-adventur...,NaN,NaN,NaN,NaN,NaN
1,e-TuBq5QTO0,2021-09-20T04:30:39Z,UCo07fumrTn1w4AxcU4j_uDw,KAKA : MERE WARGA (Official Video) Sukh-E | Ne...,Times Music,"[latest punjabi songs 2021, new punjabi song 2...",10,PT4M46S,hd,false,...,3175007,243793,4026,27774,"[https://en.wikipedia.org/wiki/Music, https://...",pa,NaN,NaN,NaN,NaN


### 2. b) Defining an empty DataFrame to ensure compatibility with the structure of all possible instances of collected data:-

In [90]:
df = pd.DataFrame(columns=["Entry_Timestamp","Video_ID","Title","Channel_Name","Channel_ID","Published_At",
                           "Title_Language","Audio_Language","Duration","Quality","Views","Likes","Dislikes",
                           "Comments","Live_Start_Real","Live_End_Real","Live_Start_Scheduled","Live_End_Scheduled",
                           "Live_Viewers","CC","Tags","Category_ID","Embeddable","Made_for_Kids","Topic_Links"])

pprint(df)

Empty DataFrame
Columns: [Entry_Timestamp, Video_ID, Title, Channel_Name, Channel_ID, Published_At, Title_Language, Audio_Language, Duration, Quality, Views, Likes, Dislikes, Comments, Live_Start_Real, Live_End_Real, Live_Start_Scheduled, Live_End_Scheduled, Live_Viewers, CC, Tags, Category_ID, Embeddable, Made_for_Kids, Topic_Links]
Index: []

[0 rows x 25 columns]


### 2. c) Renaming the columns of temporary DataFrame to properly append it to the actual empty DataFrame:-

In [91]:
print(temp_df.columns,df.columns,sep="\n")

Index(['id', 'snippet.publishedAt', 'snippet.channelId', 'snippet.title',
       'snippet.channelTitle', 'snippet.tags', 'snippet.categoryId',
       'contentDetails.duration', 'contentDetails.definition',
       'contentDetails.caption', 'status.embeddable', 'status.madeForKids',
       'statistics.viewCount', 'statistics.likeCount',
       'statistics.dislikeCount', 'statistics.commentCount',
       'topicDetails.topicCategories', 'snippet.defaultAudioLanguage',
       'snippet.defaultLanguage', 'liveStreamingDetails.actualStartTime',
       'liveStreamingDetails.actualEndTime',
       'liveStreamingDetails.scheduledStartTime'],
      dtype='object')
Index(['Entry_Timestamp', 'Video_ID', 'Title', 'Channel_Name', 'Channel_ID',
       'Published_At', 'Title_Language', 'Audio_Language', 'Duration',
       'Quality', 'Views', 'Likes', 'Dislikes', 'Comments', 'Live_Start_Real',
       'Live_End_Real', 'Live_Start_Scheduled', 'Live_End_Scheduled',
       'Live_Viewers', 'CC', 'Tags', 'Cate

In [92]:
temp_df = temp_df.rename(columns={
                                    'id':'Video_ID',
                                    'snippet.publishedAt':'Published_At','snippet.channelId':'Channel_ID','snippet.title':'Title',
                                    'snippet.channelTitle':'Channel_Name', 'snippet.tags':'Tags', 'snippet.categoryId':'Category_ID',
                                    'snippet.defaultAudioLanguage':'Audio_Language', 'contentDetails.duration':'Duration',
                                    'contentDetails.definition':'Quality', 'contentDetails.caption':'CC',
                                    'status.embeddable':'Embeddable', 'status.madeForKids':'Made_for_Kids', 'statistics.viewCount':'Views',
                                    'statistics.likeCount':'Likes', 'statistics.dislikeCount':'Dislikes',
                                    'statistics.commentCount':'Comments', 'topicDetails.topicCategories':'Topic_Links',
                                    'liveStreamingDetails.actualStartTime':'Live_Start_Real',
                                    'liveStreamingDetails.actualEndTime':'Live_End_Real',
                                    'liveStreamingDetails.scheduledStartTime':'Live_Start_Scheduled', 'snippet.defaultLanguage':'Title_Language'
})

In [93]:
temp_df.head(1)

,Video_ID,Published_At,Channel_ID,Title,Channel_Name,Tags,Category_ID,Duration,Quality,CC,...,Views,Likes,Dislikes,Comments,Topic_Links,Audio_Language,Title_Language,Live_Start_Real,Live_End_Real,Live_Start_Scheduled
0,h0vmsIhQud0,2021-09-20T06:30:04Z,UCx6F-rETGiz7xf_vkMmX2yQ,If Baburao Plays GTA 5!,Mythpat,"[gta 5 baburao, gta 5 funny video, baburao myt...",20,PT13M2S,hd,false,...,2502218,392093,4483,19455,[https://en.wikipedia.org/wiki/Action-adventur...,NaN,NaN,NaN,NaN,NaN


### 2. d) Appending temporary DataFrame to main, structured and well-defined DataFrame ( **to handle rare cases of missing values of an entire column** ):-

In [94]:
df = df.append(temp_df,ignore_index=True)

### 2. e) Adding the value for **Entry_Timestamp** column stored in the variable called ***current_timestamp*** :-

In [95]:
df["Entry_Timestamp"] = current_timestamp
print(df.iloc[0,:])

Entry_Timestamp                          2021-09-20 17:57:33.083999+00:00
Video_ID                                                      h0vmsIhQud0
Title                                             If Baburao Plays GTA 5!
Channel_Name                                                      Mythpat
Channel_ID                                       UCx6F-rETGiz7xf_vkMmX2yQ
Published_At                                         2021-09-20T06:30:04Z
Title_Language                                                        NaN
Audio_Language                                                        NaN
Duration                                                          PT13M2S
Quality                                                                hd
Views                                                             2502218
Likes                                                              392093
Dislikes                                                             4483
Comments                              

---
# 3. Data Cleansing and Data Wrangling.
---

### <li>Saving a ***`deep`*** copy of actual DataFrame to another variable *df1* [ Optional ] and **Inspecting** our data.</li>

In [96]:
df1 = df.copy()
df.head(2)

,Entry_Timestamp,Video_ID,Title,Channel_Name,Channel_ID,Published_At,Title_Language,Audio_Language,Duration,Quality,...,Live_End_Real,Live_Start_Scheduled,Live_End_Scheduled,Live_Viewers,CC,Tags,Category_ID,Embeddable,Made_for_Kids,Topic_Links
0,2021-09-20 17:57:33.083999+00:00,h0vmsIhQud0,If Baburao Plays GTA 5!,Mythpat,UCx6F-rETGiz7xf_vkMmX2yQ,2021-09-20T06:30:04Z,NaN,NaN,PT13M2S,hd,...,NaN,NaN,NaN,NaN,false,"[gta 5 baburao, gta 5 funny video, baburao myt...",20,True,False,[https://en.wikipedia.org/wiki/Action-adventur...
1,2021-09-20 17:57:33.083999+00:00,e-TuBq5QTO0,KAKA : MERE WARGA (Official Video) Sukh-E | Ne...,Times Music,UCo07fumrTn1w4AxcU4j_uDw,2021-09-20T04:30:39Z,NaN,pa,PT4M46S,hd,...,NaN,NaN,NaN,NaN,false,"[latest punjabi songs 2021, new punjabi song 2...",10,True,False,"[https://en.wikipedia.org/wiki/Music, https://..."


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   Entry_Timestamp       100 non-null    datetime64[ns, UTC]
 1   Video_ID              100 non-null    object             
 2   Title                 100 non-null    object             
 3   Channel_Name          100 non-null    object             
 4   Channel_ID            100 non-null    object             
 5   Published_At          100 non-null    object             
 6   Title_Language        15 non-null     object             
 7   Audio_Language        59 non-null     object             
 8   Duration              100 non-null    object             
 9   Quality               100 non-null    object             
 10  Views                 100 non-null    object             
 11  Likes                 100 non-null    object             
 12  Dislikes 

### 3. a) Adding calculated columns from the existing columns that contains ***`list` elements*** ( e.g.- **Tags** , **Topic_Links** ):-

1. Handling **Tags** column ( problematic ***`NaN`*** values with ***`float`*** datatype + ***List*** items with ***`list`*** datatype ).

In [98]:
# Defining a function to overcome errors caused by NaN values as they don't have len() attribute.
def count_tags(lst):
    if type(lst) is list:
        return len(lst)
    elif lst is np.nan:
        return pd.NA
    else:
        print(lst,type(lst),'This Tag item is not a list or NaN', sep='\n')

# Applying the function on "Tags" column to create "No_of_Tags" column.
df.insert(df.columns.get_loc("Tags")+1,"No_of_Tags", df["Tags"].apply(count_tags))
df["No_of_Tags"]

0        6
1       20
2       13
3        7
4     <NA>
      ... 
95      23
96      28
97      13
98      35
99      51
Name: No_of_Tags, Length: 100, dtype: object

In [99]:
# Defining a function to overcome errors caused by "[" symbol of List items.
def list_to_text(lst):
    if type(lst) is list:
        return ("{"+"}, {".join([i for i in lst])+"}")
    elif lst is np.nan:
        return pd.NA
    else:
        print(lst,type(lst),'This Tag item is not a list or NaN', sep='\n')

# Applying the function on "Tags" column to convert its items from List to Text.
# This is done to handle incompatibility between syntax of the Database, insert method chosen and python list representation with "[" symbol.
df["Tags"] = df["Tags"].apply(list_to_text)
df["Tags"]

0     {gta 5 baburao}, {gta 5 funny video}, {baburao...
1     {latest punjabi songs 2021}, {new punjabi song...
2     {american}, {prank}, {prank got failed}, {pran...
3     {nitrogen in tyres}, {helium in tyres}, {flyin...
4                                                  <NA>
                            ...                        
95    {#kanimangalam kovilakam}, {#malayalam web ser...
96    {raunak}, {rj raunak}, {raunak nopo}, {rj raun...
97    {food challenge india}, {hungry birds channel}...
98    {Chacha ji aur girlfriend}, {Funny chacha ji},...
99    {khushi}, {khushipunjaban}, {khushichoudhary},...
Name: Tags, Length: 100, dtype: object

2. Handling **Topic_Links** column ( problematic ***`NaN`*** values with ***`float`*** datatype + ***List*** items with ***`list`*** datatype and unwanted *URL* parts).

In [100]:
# Defining a function to overcome errors caused by NaN values.
def get_links(lst):
    if type(lst) is list:
        return (", ".join([i.replace("https://en.wikipedia.org/wiki/","") for i in lst]))
    elif lst is np.nan:
        return pd.NA
    else:
        print(lst,type(lst),'This Topic_Links item is not a list or NaN', sep='\n')

# Applying the function on "Topic_Links" column to create "Topics" column.
df["Topics"] = df["Topic_Links"].apply(get_links)
df["Topics"]

0     Action-adventure_game, Action_game, Video_game...
1                                  Music, Music_of_Asia
2                           Food, Lifestyle_(sociology)
3                                               Vehicle
4                                         Entertainment
                            ...                        
95              Entertainment, Film, Television_program
96                                       Cricket, Sport
97                                Lifestyle_(sociology)
98                                  Entertainment, Film
99                                Lifestyle_(sociology)
Name: Topics, Length: 100, dtype: object

3. Adding the **Rank** column to later perform **detailed** analysis of popular videos of different datetimes.

In [101]:
df.insert(0,"Rank",range(1,101))

4. Dropping **unwanted** / **redundant** columns ( e.g.- **Topic_Links** ) that was earlier required to get the calculated columns.

In [102]:
df.drop(columns=["Topic_Links"],inplace=True)

5. Noting down the mean **memory usage** per column of our DataFrame.

In [103]:
memory_1 = round(df.memory_usage(deep=True).sum()/df.columns.size)
memory_1

6898

### 3. b) Transforming data-formats ( e.g.- ***time-format***, ***duration-format***, ***language-format*** ) and data-types ( e.g.- ***`strings`*** ) to appropriate and interpretable format:-

1. Parsing columns containing Date-Time information, as timezone-aware ***`datetime.datetime`*** object for easier access to its details of *date*, *month*, *year*, *hour*, *minute*, *second* and *time-zone*.

In [104]:
df["Published_At"]          = pd.to_datetime(df["Published_At"],errors="coerce",utc=True)
df["Live_Start_Real"]       = pd.to_datetime(df["Live_Start_Real"],errors="coerce",utc=True)
df["Live_End_Real"]         = pd.to_datetime(df["Live_End_Real"],errors="coerce",utc=True)
df["Live_Start_Scheduled"]  = pd.to_datetime(df["Live_Start_Scheduled"],errors="coerce",utc=True)
df["Live_End_Scheduled"]    = pd.to_datetime(df["Live_End_Scheduled"],errors="coerce",utc=True)

2. Decoding the YouTube language codes [ [i18nLanguages Documentation](https://developers.google.com/youtube/v3/docs/i18nLanguages "YouTube Supported Codes for Languages") ] of **Title_Language** and **Audio_Language** columns.

2. 
    1. Preparing the reference table / DataFrame for Language codes using different YouTube api.

In [105]:
lang_url = "https://youtube.googleapis.com/youtube/v3/i18nLanguages?part=snippet&prettyPrint=true&fields=items%2Fsnippet&key="+api_key
lang_data = requests.get(lang_url).json()
lang_df = pd.json_normalize(lang_data["items"])
lang_df.columns = ["Language_Code","Language"]
lang_df = lang_df.convert_dtypes()
print(lang_df.nunique(),sep='\n')
lang_df.head()

Language_Code    83
Language         83
dtype: int64


,Language_Code,Language
0,af,Afrikaans
1,am,Amharic
2,ar,Arabic
3,as,Assamese
4,az,Azerbaijani


2. 
    2. Decoding columns using refence DataFrame.

In [106]:
temp_df = df[["Title_Language"]].merge(lang_df,how="left",left_on="Title_Language",right_on="Language_Code")["Language"]
df.insert(df.columns.get_loc("Title_Language")+1,"Title_Language_Name",temp_df)
temp_df = df[["Audio_Language"]].merge(lang_df,how="left",left_on="Audio_Language",right_on="Language_Code")["Language"]
df.insert(df.columns.get_loc("Audio_Language")+1,"Audio_Language_Name",temp_df)

2. 
    3. Assigning fixed Language code ***"zxx"*** to all **out of reference** language codes ( [ISO639 Documentation](https://iso639-3.sil.org/code/zxx "ISO 639-3 Identifier Documentation: zxx") ).

In [107]:
df.loc[(df["Title_Language"].notnull()) & (df["Title_Language_Name"].isnull()),["Title_Language"]] = "zxx"
df.loc[(df["Title_Language"].notnull()) & (df["Title_Language_Name"].isnull()),["Title_Language_Name"]] = "Not Applicable"
df.loc[(df["Audio_Language"].notnull()) & (df["Audio_Language_Name"].isnull()),["Audio_Language"]] = "zxx"
df.loc[(df["Audio_Language"].notnull()) & (df["Audio_Language_Name"].isnull()),["Audio_Language_Name"]] = "Not Applicable"

3. Inspecting the structure of the DataFrame to start converting datatypes if preferrable and possible.

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   Rank                  100 non-null    int32              
 1   Entry_Timestamp       100 non-null    datetime64[ns, UTC]
 2   Video_ID              100 non-null    object             
 3   Title                 100 non-null    object             
 4   Channel_Name          100 non-null    object             
 5   Channel_ID            100 non-null    object             
 6   Published_At          100 non-null    datetime64[ns, UTC]
 7   Title_Language        15 non-null     object             
 8   Title_Language_Name   15 non-null     string             
 9   Audio_Language        59 non-null     object             
 10  Audio_Language_Name   59 non-null     string             
 11  Duration              100 non-null    object             
 12  Quality  

4. Automatically converting column datatypes to best possible datatypes, supporting ***`pd.NA `***.

In [109]:
df = df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   Rank                  100 non-null    Int32              
 1   Entry_Timestamp       100 non-null    datetime64[ns, UTC]
 2   Video_ID              100 non-null    string             
 3   Title                 100 non-null    string             
 4   Channel_Name          100 non-null    string             
 5   Channel_ID            100 non-null    string             
 6   Published_At          100 non-null    datetime64[ns, UTC]
 7   Title_Language        15 non-null     string             
 8   Title_Language_Name   15 non-null     string             
 9   Audio_Language        59 non-null     string             
 10  Audio_Language_Name   59 non-null     string             
 11  Duration              100 non-null    string             
 12  Quality  

5. Parsing **Duration** column as python ***`datetime.timedelta`*** object type.</br>
( This is intentionally done after using ***`convert_dtypes`***. Otherwise, the ***`timedelta64[ns]`*** datatype will also be converted to unwanted, very large ( timedelta as in ***nano-second*** scale ) ***`float64`*** datatype. )

In [110]:
df["Duration"] = df["Duration"].apply(pd.Timedelta)

6. Changing the ***`string`*** datatype of ***boolean*** column ( e.g.- **CC** ) to ***`bool`*** datatype:-

In [111]:
df["CC"] = df["CC"].map({"true":True,"false":False})

7. Changing the ***`string`*** datatype of ***numeric*** columns ( e.g.- **Views** , **Likes** ) to ***`integer`*** datatype:-

In [112]:
df.loc[:,"Views":"Comments"] = df.loc[:,"Views":"Comments"].astype("Int64")
df["No_of_Tags"] = df["No_of_Tags"].astype("UInt16")

8. Modifying the ***`Integer`*** datatypes to **memory-efficient** integer datatypes.

In [113]:
df["Live_Viewers"] = df["Live_Viewers"].astype("UInt32")
df["Rank"] = df["Rank"].astype("UInt8")

9. Converting suitable columns ( e.g.- **Quality**, **Audio_Language** ) to memory efficient ***`categorical`*** datatype.

In [114]:
df["Quality"] = df["Quality"].astype('category')
df.loc[:,"Title_Language":"Audio_Language_Name"] = df.loc[:,"Title_Language":"Audio_Language_Name"].astype('category')
df["Category_ID"] = df["Category_ID"].astype('category')

10. Noting down the mean **data usage** per column of the final, processed DataFrame.

In [115]:
memory_2 = round(df.memory_usage(deep=True).sum()/df.columns.size)
memory_2

3652

### 3. c) Calculating **memory efficiency** obtained by processing the DataFrame:-

In [116]:
memory_efficiency = round((memory_1-memory_2)*100/memory_1)
print(f"----- Total memory usage is reduced by {memory_efficiency}% -----")

----- Total memory usage is reduced by 47% -----


---
# 4. Sending Data to remote PostgreSQL database on Compute Engine of Google Cloud Platform ( *Using only free tier products* ).
---

<p>In all practical cases, it is common to store the collected data after cleaning as historical data in a database like a Data-Warehouse. And then fetch the required data directly from the database anytime for analysis purposes. This allows us to analize data that were collected at different timestamps and discover trends and patterns in our data. Now, as our data is already cleaned, it is ready and waiting to be sent to the Data-Warehouse.</br>
The reason for choosing a <b>Cloud Database</b> over local database is that, due to increasing volume of data, it is better to shift to cloud for easier handling of all loads and also running local server <b>24*7</b> is not possible for most of the individuals.</p>

### 4. a) Loading Database credentials from ***`.env `*** file:-

In [117]:
host_name = os.getenv("HOST_NAME")
db_name = os.getenv("DB_NAME")
port = os.getenv("PORT")
user_name = os.getenv("USER_NAME")
password = os.getenv("PASSWORD")

### 4. b) Connecting to **Cloud Database** using ***`psycopg2 `*** and creating the ***`cursor()`*** for executing **SQL** queries:-

In [118]:
# Establishing connection.
try:
    connection = spg.connect(host=host_name, database=db_name, port=port, user=user_name, password=password)
except spg.OperationalError as error:
    raise error
else:
    print("Successfully connected to the PostgreSQL database!")

# Creating cursor.
cursor = connection.cursor()

Successfully connected to the PostgreSQL database!


### 4. c) Creating table in the database:-

1. Creating PostgreSQL **`ENUM DATATYPE` s;** required ( [See Documentation](https://www.postgresql.org/docs/12/datatype-enum.html "PostgreSQL Official Documentation") ) for assigning **Enumerated** datatypes to table columns.

In [119]:
%%script echo 'Skipping executing codes of this cell as it is to be run only once while other cells are to be run everytime to collect data from API.'
# Please comment out above line to execute all following codes of this cell.
# But again uncomment it to stop executing below codes as otherwise the database will throw an error.

# Creating tuples required to define enum columns.
lang_enum      = tuple(lang_df.Language_Code.values)
quality_enum   = ("sd","hd")
lang_name_enum = tuple(lang_df.Language.values)

# Writing SQL Queries.
create_type_lang      = (f"CREATE TYPE LANG AS ENUM {lang_enum};"
                          "ALTER TYPE LANG ADD VALUE 'zxx'")                    # Adding the ISO639 specified code "zxx" in our enum type.
create_type_lang_name = (f"CREATE TYPE LANG_NAME AS ENUM {lang_name_enum};"
                          "ALTER TYPE LANG_NAME ADD VALUE 'Not Applicable'")    # Adding the interpretation of code "zxx" in our enum type.
create_type_quality   = (f"CREATE TYPE QUALITY AS ENUM {quality_enum}")

# Executing Queries in the database.
try:
    cursor.execute(create_type_lang)
    cursor.execute(create_type_lang_name)
    cursor.execute(create_type_quality)
    cursor.execute("COMMIT")
except (Exception, spg.DatabaseError) as error:
    print(f"Following error(s) occured : {error}")
    cursor.execute("ROLLBACK")

Skipping executing codes of this cell as it is to be run only once while other cells are to be run everytime to collect data from API.


2. Creating empty database table with columns assigned proper datatypes.

In [120]:
# Writing SQL command.
create_table = ("""CREATE TABLE IF NOT EXISTS 
                   YT_POPULAR_VIDEOS (
                      
                      RANK                   SMALLINT     NOT NULL,
                      ENTRY_TIMESTAMP        TIMESTAMPTZ  NOT NULL DEFAULT CURRENT_TIMESTAMP,
                      VIDEO_ID               VARCHAR(255) NOT NULL,
                      TITLE                  VARCHAR(255) NOT NULL,
                      CHANNEL_NAME           VARCHAR(255) NOT NULL,
                      CHANNEL_ID             VARCHAR(255) NOT NULL,
                      PUBLISHED_AT           TIMESTAMPTZ  NOT NULL,
                      TITLE_LANGUAGE         LANG,
                      TITLE_LANGUAGE_NAME    LANG_NAME,
                      AUDIO_LANGUAGE         LANG,
                      AUDIO_LANGUAGE_NAME    LANG_NAME,
                      DURATION               INTERVAL,
                      QUALITY                QUALITY      NOT NULL,
                      VIEWS                  BIGINT,
                      LIKES                  BIGINT,
                      DISLIKES               BIGINT,
                      COMMENTS               BIGINT,
                      LIVE_START_REAL        TIMESTAMPTZ,
                      LIVE_END_REAL          TIMESTAMPTZ,
                      LIVE_START_SCHEDULED   TIMESTAMPTZ,
                      LIVE_END_SCHEDULED     TIMESTAMPTZ,
                      LIVE_VIEWERS           INTEGER,
                      CC                     BOOLEAN,
                      TAGS                   TEXT,
                      NO_OF_TAGS             INTEGER,
                      CATEGORY_ID            VARCHAR(255),
                      EMBEDDABLE             BOOLEAN,
                      MADE_FOR_KIDS          BOOLEAN,
                      TOPICS                 VARCHAR(255),

                      CONSTRAINT PK_VID_ENTRY PRIMARY KEY (VIDEO_ID,ENTRY_TIMESTAMP)
                   );
                
                """)
# Executing the command.
try:
   cursor.execute(create_table)
   cursor.execute("COMMIT")
except (Exception, spg.DatabaseError) as error:
    print(f"Following error(s) occured : {error}")
    cursor.execute("ROLLBACK")

### 4. d) Using the fastest method ***`copy_from()`*** using ***`StringIO`*** to insert data into database table from our cleaned DataFrame.

In [121]:
buffer = StringIO()

# Storing the DataFrame to StringIO in CSV format.
df.to_csv(buffer, sep="\t", header=False, index=False) 

# seek(0) will position the pointer to the start of the data, 0th position/byte.
buffer.seek(0)

# Executing the copy to load DataFrame data to the table in database.
try:
    cursor.copy_from(buffer,'yt_popular_videos',sep="\t",null="")
    cursor.execute("COMMIT")
except (Exception, spg.DatabaseError) as error:
    print(f"Following error(s) occured : {error}")
    cursor.execute("ROLLBACK")

### 4. e) **Closing** the ***`cursor()`*** and ***`connection()`*** to the database.

In [122]:
cursor.close()
connection.close()

### 4. f) **Clearing** all the user defined variables of the namespace.

In [123]:
%reset -f